In [1]:
import numpy as np
import SimpleITK as sitk
import os 
import glob

In [2]:
# 分别为原始图像及其分割结果的路径
image_path = r'D:/Transmorph_Frame/Data/raw_images/new_nii_files/new_nii_resampled_files/'
tumor_path = r'D:/Transmorph_Frame/Data/raw_tumors_seg/new_nii_files/new_nii_resampled_files/'
os.makedirs(image_path + r'new_nii_resampled_cropped_files', exist_ok=True)
image_path_new = image_path + r'new_nii_resampled_cropped_files/'
os.makedirs(tumor_path + r'new_nii_resampled_cropped_files', exist_ok=True)
tumor_path_new = tumor_path + r'new_nii_resampled_cropped_files/'

In [3]:
# 按照CT和MR的分割结果确定crop的中心，方便后续配准
CT_tumors = glob.glob(tumor_path + '*CT*.nii.gz')
MR_tumors = glob.glob(tumor_path + '*MR*.nii.gz')
CT_images = glob.glob(image_path + '*CT*.nii.gz')
MR_images = glob.glob(image_path + '*MR*.nii.gz')

In [4]:
print(CT_tumors)
print(MR_tumors)
print(CT_images)
print(MR_images)

['D:/Transmorph_Frame/Data/raw_tumors_seg/new_nii_files/new_nii_resampled_files\\PLA-2001CT_resampled.nii.gz', 'D:/Transmorph_Frame/Data/raw_tumors_seg/new_nii_files/new_nii_resampled_files\\PLA-2002CT_resampled.nii.gz', 'D:/Transmorph_Frame/Data/raw_tumors_seg/new_nii_files/new_nii_resampled_files\\PLA-2004CT_resampled.nii.gz', 'D:/Transmorph_Frame/Data/raw_tumors_seg/new_nii_files/new_nii_resampled_files\\PLA-2007CT_resampled.nii.gz', 'D:/Transmorph_Frame/Data/raw_tumors_seg/new_nii_files/new_nii_resampled_files\\PLA-2010CT_resampled.nii.gz', 'D:/Transmorph_Frame/Data/raw_tumors_seg/new_nii_files/new_nii_resampled_files\\PLA-2013CT_resampled.nii.gz', 'D:/Transmorph_Frame/Data/raw_tumors_seg/new_nii_files/new_nii_resampled_files\\PLA-2021CT_resampled.nii.gz', 'D:/Transmorph_Frame/Data/raw_tumors_seg/new_nii_files/new_nii_resampled_files\\PLA-2024CT_resampled.nii.gz', 'D:/Transmorph_Frame/Data/raw_tumors_seg/new_nii_files/new_nii_resampled_files\\PLA-2028CT_resampled.nii.gz', 'D:/Trans

In [5]:
# 使用一个列表存储CT和MR的估计中心位置
CT_indexes_list, MR_indexes_list = [], []
for path in CT_tumors:
    tumor = sitk.ReadImage(path)
    tumor_array = sitk.GetArrayFromImage(tumor)
    indexes = np.array(np.where(tumor_array == 1))
    CT_indexes_list.append(np.mean(indexes, axis=1).astype(int))
for path in MR_tumors:
    tumor = sitk.ReadImage(path)
    tumor_array = sitk.GetArrayFromImage(tumor)
    indexes = np.array(np.where(tumor_array == 1))
    MR_indexes_list.append(np.mean(indexes, axis=1).astype(int)) 

In [6]:
print(CT_indexes_list)
print(MR_indexes_list)

[array([ 46,  88, 133]), array([ 71,  83, 128]), array([ 22,  99, 184]), array([ 61, 101, 162]), array([75, 86, 74]), array([ 63, 131, 207]), array([ 46, 123, 161]), array([ 69, 146, 166]), array([ 45, 111, 193]), array([ 29, 148, 114]), array([ 70, 115, 188]), array([ 79, 108, 200]), array([ 88,  94, 154]), array([ 72, 141,  80]), array([46, 51, 60]), array([ 76, 108, 163]), array([ 97,  33, 126]), array([ 42, 128, 219]), array([ 75, 123,  42]), array([ 66,  65, 107]), array([ 94,  81, 185]), array([ 75,  91, 120]), array([ 51, 132, 120]), array([ 88,  98, 103]), array([78, 83, 91]), array([102,  81, 146]), array([177,  39, 105]), array([ 97,  66, 133]), array([ 45, 103,  70]), array([ 52, 137,  68]), array([183,  75,  76]), array([87, 45, 84]), array([ 86,  72, 128]), array([165,  74, 110]), array([53, 97, 96]), array([147,  43, 121]), array([ 36,  95, 130]), array([153,  85, 119]), array([21, 84, 60]), array([ 47,  83, 103]), array([ 78, 107,  86]), array([ 44,  68, 128]), array([69

In [7]:
def pad_axis_score(arr, axis, pad_size, is_center=True, forward_or_back=True):
    if axis == 0:
        if is_center:
            # 是否沿着中心进行扩充
            return np.pad(arr, pad_width=((pad_size + 1, pad_size + 1), (0, 0), (0, 0)))
        else:
            if forward_or_back:
                # 沿着前一个方向进行扩充,此时由于没有取整操作，不需要进行加一
                return np.pad(arr, pad_width=((pad_size + 1, 0), (0, 0), (0, 0)))
            else:
                return np.pad(arr, pad_width=((0, pad_size + 1), (0, 0), (0, 0)))
    elif axis == 1:
        if is_center:
            # 是否沿着中心进行扩充
            return np.pad(arr, pad_width=((0, 0), (pad_size + 1, pad_size + 1), (0, 0)))
        else:
            if forward_or_back:
                # 沿着前一个方向进行扩充,此时由于没有取整操作，不需要进行加一
                return np.pad(arr, pad_width=((0, 0), (pad_size + 1, 0), (0, 0)))
            else:
                return np.pad(arr, pad_width=((0, 0), (0, pad_size + 1), (0, 0)))
    elif axis == 2:    
        if is_center:
            # 是否沿着中心进行扩充
            return np.pad(arr, pad_width=((0, 0), (0, 0), (pad_size + 1, pad_size + 1)))
        else:
            if forward_or_back:
                # 沿着前一个方向进行扩充,此时由于没有取整操作，不需要进行加一
                return np.pad(arr, pad_width=((0, 0), (0, 0), (pad_size + 1, 0)))
            else:
                return np.pad(arr, pad_width=((0, 0), (0, 0), (0, pad_size + 1)))
    else:
        print('请确保axis的值在0-2之间')

In [8]:
CT_path, MR_path = CT_tumors, MR_tumors
CT_path_2, MR_path_2 = CT_images, MR_images
CT_indexes, MR_indexes = CT_indexes_list, MR_indexes_list
new_shape = [96, 160, 160]
for ct_path, mr_path,ct_path_2, mr_path_2, ct_index, mr_index in zip(CT_path, MR_path, CT_path_2, MR_path_2, CT_indexes, MR_indexes):
    # 读取相应的肿瘤数据
    ct_image = sitk.ReadImage(ct_path)
    mr_image = sitk.ReadImage(mr_path)
    # 读取相应的图像数据
    ct_image_2 = sitk.ReadImage(ct_path_2)
    mr_image_2 = sitk.ReadImage(mr_path_2)
    # 转换为numpy格式
    ct_array = sitk.GetArrayFromImage(ct_image)
    mr_array = sitk.GetArrayFromImage(mr_image)
    ct_array_2 = sitk.GetArrayFromImage(ct_image_2)
    mr_array_2 = sitk.GetArrayFromImage(mr_image_2)
    # 读取对应的形状大小，肿瘤和图像进行同等操作，因此只需读取一次
    ct_shape = ct_array.shape
    mr_shape = mr_array.shape
    # 进行大小判断，如果大小过小，则在需要进行补零操作，采取的方案是直接两侧补零
    # 首先是CT大小的判断，需要进行对称操作
    # 存储偏移量
    ct_pad_size, mr_pad_size = np.array([0, 0, 0]), np.array([0, 0, 0])
    for i in range(len(new_shape)):
        ct_pad_size[i] = int((new_shape[i] - ct_shape[i]) / 2) * (ct_shape[i] < new_shape[i])
        mr_pad_size[i] = int((new_shape[i] - mr_shape[i]) / 2) * (mr_shape[i] < new_shape[i])
        ct_array = pad_axis_score(ct_array, i, ct_pad_size[i])
        ct_array_2 = pad_axis_score(ct_array_2, i, ct_pad_size[i])
        # 对称操作
        mr_array = pad_axis_score(mr_array, i, mr_pad_size[i])
        mr_array_2 = pad_axis_score(mr_array_2, i, mr_pad_size[i])
    # 更新新的形状
    ct_shape = ct_array.shape
    mr_shape = mr_array.shape
    # 利用偏移量对index进行修正
    ct_index = ct_index + ct_pad_size
    mr_index = mr_index + mr_pad_size
    axis_bias = [0, 0, 0]
    for i in range(len(new_shape)):
        true_bias = np.array([ct_index[i],
                            ct_shape[i] - ct_index[i],
                            mr_index[i],
                            mr_shape[i] - mr_index[i]])
        axis_bias[i] = true_bias
    # 记录三个轴不同方向的最小距离
    bias_min_index = [np.argmin(axis_bias[0]), np.argmin(axis_bias[1]), np.argmin(axis_bias[2])]
    # 取值的范围定义
    ct_range, mr_range = [0, 0, 0], [0, 0, 0]
    # 判断是否需要置零的指标
    ct_bias_flag, mr_bias_flag = [False, False, False], [False, False, False]
    # 定义ct的负数偏移，定义ct的正向偏移
    ct_bias_nag, ct_bias_pos = [0, 0, 0], [0, 0, 0]
    mr_bias_nag, mr_bias_pos = [0, 0, 0], [0, 0, 0]
    for i in range(len(new_shape)):
        # z方向进行判断
        if bias_min_index[i] == 0:
            ct_range[i] = range(0, new_shape[i])
            mr_range[i] = range(mr_index[i] - axis_bias[i][bias_min_index[i]], mr_index[i] - axis_bias[i][bias_min_index[i]] + new_shape[i])
            # 差距过大越界的判断
            if mr_range[i][0] < 0:
                mr_bias_flag[i] = True
                mr_bias_nag[i] = -mr_range[i][0]
                ct_array = pad_axis_score(ct_array, i, -mr_range[i][0], is_center=False)
                ct_array_2 = pad_axis_score(ct_array_2, i, -mr_range[i][0], is_center=False)
                # 对称操作
                mr_array = pad_axis_score(mr_array, i, -mr_range[i][0], is_center=False)
                mr_array_2 = pad_axis_score(mr_array_2, i, -mr_range[i][0], is_center=False)
                mr_range[i] = range(0, new_shape[i])
            if mr_range[i][-1] >= mr_shape[i]:
                mr_bias_flag[i] = True
                mr_bias_pos[i] = mr_range[i][-1] - mr_shape[i]
                ct_array = pad_axis_score(ct_array, i, mr_range[i][-1] - mr_shape[i], is_center=False, forward_or_back=False)
                ct_array_2 = pad_axis_score(ct_array_2, i, mr_range[i][-1] - mr_shape[i], is_center=False, forward_or_back=False)
                mr_array = pad_axis_score(mr_array, i, mr_range[i][-1] - mr_shape[i], is_center=False, forward_or_back=False)
                mr_array_2 = pad_axis_score(mr_array_2, i, mr_range[i][-1] - mr_shape[i], is_center=False, forward_or_back=False)
                new_range_max = mr_range[i][-1]
                mr_range[i] = range(new_range_max - new_shape[i], new_range_max)
        elif bias_min_index[i] == 1:
            ct_range[i] = range(ct_shape[i] - new_shape[i], ct_shape[i])
            mr_range[i] = range(mr_index[i] + axis_bias[i][bias_min_index[i]] - new_shape[i], mr_index[i] + axis_bias[i][bias_min_index[i]])
            # 差距过大越界的判断
            if mr_range[i][0] < 0:
                mr_bias_flag[i] = True
                mr_bias_nag[i] = -mr_range[i][0]
                ct_array = pad_axis_score(ct_array, i, -mr_range[i][0], is_center=False)
                ct_array_2 = pad_axis_score(ct_array_2, i, -mr_range[i][0], is_center=False)
                mr_array = pad_axis_score(mr_array, i, -mr_range[i][0], is_center=False)
                mr_array_2 = pad_axis_score(mr_array_2, i, -mr_range[i][0], is_center=False)
                mr_range[i] = range(0, new_shape[i])
            if mr_range[i][-1] >= mr_shape[i]:
                mr_bias_flag[i] = True
                mr_bias_pos[i] = mr_range[i][-1] - mr_shape[i]
                ct_array = pad_axis_score(ct_array, i, mr_range[i][-1] - mr_shape[i], is_center=False, forward_or_back=False)
                ct_array_2 = pad_axis_score(ct_array_2, i, mr_range[i][-1] - mr_shape[i], is_center=False, forward_or_back=False)
                mr_array = pad_axis_score(mr_array, i, mr_range[i][-1] - mr_shape[i], is_center=False, forward_or_back=False) 
                mr_array_2 = pad_axis_score(mr_array_2, i, mr_range[i][-1] - mr_shape[i], is_center=False, forward_or_back=False) 
                new_range_max = mr_range[i][-1]
                mr_range[i] = range(new_range_max - new_shape[i], new_range_max)
        elif bias_min_index[i] == 2:
            ct_range[i] = range(ct_index[i] - axis_bias[i][bias_min_index[i]], ct_index[i] - axis_bias[i][bias_min_index[i]] + new_shape[i])
            mr_range[i] = range(0, new_shape[i])
            # 差距过大越界的判断
            if ct_range[i][0] < 0:
                ct_bias_flag[i] = True
                ct_bias_nag[i] = -ct_range[i][0]
                ct_array = pad_axis_score(ct_array, i, -ct_range[i][0], is_center=False)
                ct_array_2 = pad_axis_score(ct_array_2, i, -ct_range[i][0], is_center=False)
                mr_array = pad_axis_score(mr_array, i, -ct_range[i][0], is_center=False)
                mr_array_2 = pad_axis_score(mr_array_2, i, -ct_range[i][0], is_center=False)
                ct_range[i] = range(0, new_shape[i])
            if ct_range[i][-1] >= ct_shape[i]:
                ct_bias_flag[i] = True
                ct_bias_pos[i] = ct_range[i][-1] - ct_shape[i]
                ct_array = pad_axis_score(ct_array, i, ct_range[i][-1] - ct_shape[i], is_center=False, forward_or_back=False)
                ct_array_2 = pad_axis_score(ct_array_2, i, ct_range[i][-1] - ct_shape[i], is_center=False, forward_or_back=False)
                mr_array = pad_axis_score(mr_array, i, ct_range[i][-1] - ct_shape[i], is_center=False, forward_or_back=False) 
                mr_array_2 = pad_axis_score(mr_array_2, i, ct_range[i][-1] - ct_shape[i], is_center=False, forward_or_back=False) 
                new_range_max = ct_range[i][-1]
                ct_range[i] = range(new_range_max - new_shape[i], new_range_max)
        else:
            ct_range[i] = range(ct_index[i] + axis_bias[i][bias_min_index[i]] - new_shape[i], ct_index[i] + axis_bias[i][bias_min_index[i]])
            mr_range[i] = range(mr_shape[i] - new_shape[i], mr_shape[i])
            # 差距过大越界的判断
            if ct_range[i][0] < 0:
                ct_bias_flag[i] = True
                ct_bias_nag[i] = -ct_range[i][0]
                ct_array = pad_axis_score(ct_array, i, -ct_range[i][0], is_center=False)
                ct_array_2 = pad_axis_score(ct_array_2, i, -ct_range[i][0], is_center=False)
                # 对称操作
                mr_array = pad_axis_score(mr_array, i, -ct_range[i][0], is_center=False) 
                mr_array_2 = pad_axis_score(mr_array_2, i, -ct_range[i][0], is_center=False) 
                ct_range[i] = range(0, new_shape[i])
            if ct_range[i][-1] >= ct_shape[i]:
                ct_bias_flag[i] = True
                ct_bias_pos[i] = ct_range[i][-1] - ct_shape[i]
                ct_array = pad_axis_score(ct_array, i, ct_range[i][-1] - ct_shape[i], is_center=False, forward_or_back=False)
                ct_array_2 = pad_axis_score(ct_array_2, i, ct_range[i][-1] - ct_shape[i], is_center=False, forward_or_back=False)
                mr_array = pad_axis_score(mr_array, i, ct_range[i][-1] - ct_shape[i], is_center=False, forward_or_back=False) 
                mr_array_2 = pad_axis_score(mr_array_2, i, ct_range[i][-1] - ct_shape[i], is_center=False, forward_or_back=False) 
                new_range_max = ct_range[i][-1]
                ct_range[i] = range(new_range_max - new_shape[i], new_range_max)
    # print(f'取得范围为{ct_range[0], ct_range[1], ct_range[2]}')
    # print(ct_array.shape)
    # print(f'取得范围为{mr_range[0], mr_range[1], mr_range[2]}')
    ct_new_array = ct_array[ct_range[0], :, :][:, ct_range[1], :][:, :, ct_range[2]]
    ct_new_array_2 = ct_array_2[ct_range[0], :, :][:, ct_range[1], :][:, :, ct_range[2]]
    mr_new_array = mr_array[mr_range[0], :, :][:, mr_range[1], :][:, :, mr_range[2]]
    mr_new_array_2 = mr_array_2[mr_range[0], :, :][:, mr_range[1], :][:, :, mr_range[2]]
    # 利用numpy获取需要的偏移量
    ct_bias_flag, mr_bias_flag = np.array(ct_bias_flag), np.array(mr_bias_flag)
    ct_bias_index = np.where(ct_bias_flag==True)[0]
    mr_bias_index = np.where(mr_bias_flag==True)[0]
    if len(ct_bias_index):
        for index in ct_bias_index:
            if index == 0:
                # 置零
                mr_new_array[0:ct_bias_nag[index], :, :] = 0
                mr_new_array[new_shape[index]-ct_bias_pos[index]:new_shape[index], :,:] = 0
                mr_new_array_2[0:ct_bias_nag[index], :, :] = 0
                mr_new_array_2[new_shape[index]-ct_bias_pos[index]:new_shape[index], :,:] = 0
            elif index == 1:
                mr_new_array[:, 0:ct_bias_nag[index], :] = 0
                mr_new_array[:, new_shape[index]-ct_bias_pos[index]:new_shape[index], :] = 0
                mr_new_array_2[:, 0:ct_bias_nag[index], :] = 0
                mr_new_array_2[:, new_shape[index]-ct_bias_pos[index]:new_shape[index], :] = 0
            else:
                mr_new_array[:, :, 0:ct_bias_nag[index]] = 0
                mr_new_array[:, :, new_shape[index]-ct_bias_pos[index]:new_shape[index]] = 0
                mr_new_array_2[:, :, 0:ct_bias_nag[index]] = 0
                mr_new_array_2[:, :, new_shape[index]-ct_bias_pos[index]:new_shape[index]] = 0
    elif len(mr_bias_index):
        for index in mr_bias_index:
            if index == 0:
                # 置零
                ct_new_array[0:mr_bias_nag[index], :, :] = 0
                ct_new_array[new_shape[index]-mr_bias_pos[index]:new_shape[index], :,:] = 0
                ct_new_array_2[0:mr_bias_nag[index], :, :] = 0
                ct_new_array_2[new_shape[index]-mr_bias_pos[index]:new_shape[index], :,:] = 0
            elif index == 1:
                ct_new_array[:, 0:mr_bias_nag[index], :] = 0
                ct_new_array[:, new_shape[index]-mr_bias_pos[index]:new_shape[index], :] = 0
                ct_new_array_2[:, 0:mr_bias_nag[index], :] = 0
                ct_new_array_2[:, new_shape[index]-mr_bias_pos[index]:new_shape[index], :] = 0
            else:
                ct_new_array[:, :, 0:mr_bias_nag[index]] = 0
                ct_new_array[:, :, new_shape[index]-mr_bias_pos[index]:new_shape[index]] = 0
                ct_new_array_2[:, :, 0:mr_bias_nag[index]] = 0
                ct_new_array_2[:, :, new_shape[index]-mr_bias_pos[index]:new_shape[index]] = 0
    ct_new_image = sitk.GetImageFromArray(ct_new_array)
    mr_new_image = sitk.GetImageFromArray(mr_new_array)
    ct_new_image_2 = sitk.GetImageFromArray(ct_new_array_2)
    mr_new_image_2 = sitk.GetImageFromArray(mr_new_array_2)
    sitk.WriteImage(ct_new_image, ct_path.replace('new_nii_resampled_files', 'new_nii_resampled_files/new_nii_resampled_cropped_files')
                    .replace('_resampled.nii.gz', '_resampled_cropped.nii.gz'))
    sitk.WriteImage(mr_new_image, mr_path.replace('new_nii_resampled_files', 'new_nii_resampled_files/new_nii_resampled_cropped_files')
                    .replace('_resampled.nii.gz', '_resampled_cropped.nii.gz'))   
    sitk.WriteImage(ct_new_image_2, ct_path_2.replace('new_nii_resampled_files', 'new_nii_resampled_files/new_nii_resampled_cropped_files')
                    .replace('_resampled.nii.gz', '_resampled_cropped.nii.gz'))
    sitk.WriteImage(mr_new_image_2, mr_path_2.replace('new_nii_resampled_files', 'new_nii_resampled_files/new_nii_resampled_cropped_files')
                    .replace('_resampled.nii.gz', '_resampled_cropped.nii.gz'))     